In [1]:
#%%
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "pass"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31820
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, HOST,PORT , DB , COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GS_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('Shawn Neal - Project Two - CS-340 Dashboard'))),
    html.Center(html.Img(id='Grazioso Salvare Logo',src='data:image/png;base64,{}'.
                         format(encoded_image.decode()),alt='Grazioso Salvare Logo')),
    html.Hr(),
    html.Div([
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        html.H4(html.Center(html.Label(['Select A Filter Option: '], style={'font-weight': 'italics'}))),
        html.H4(html.Center(dcc.RadioItems(
            id='filter_type',
            options=[
                {'label': 'Water Rescue    ', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue    ', 'value': 'mount'},
                {'label': 'Disaster/Individual Tracking    ', 'value': 'disaster'},
                {'label': 'Reset    ', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        ))
    )]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        
        data=df.to_dict('records'),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=12,

    ),
    html.Br(),
    html.Hr(),
    # This div sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Center(html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                         
                         dcc.Graph(
                             id='graph-id',
                             className='col s12 m6',
                        style={
                        "width": "500px",
                        "height": "500px",
                        "display": "inline-block",
                        "padding-top": "5px",
                        "padding-left": "50px",
                            }
                         ),
                       
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                        style={
                        "width": "500px",
                        "height": "500px",
                        "display": "inline-block",
                        "padding-top": "5px",
                        "padding-left": "50px",
                            }
                         ),
                     ]
                 ),
             ])
        )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

# Radio button filters
@app.callback(Output('datatable-id','data'),
              [Input('filter_type', 'value')])

def update_table(filter_selection):
    
        #Filter for water rescue
        if filter_selection == 'water':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Filter for Mountain/Wilderness Rescue
        elif filter_selection == 'mount':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Filter for Disaster Rescue/Individual Tracking
        elif filter_selection == 'disaster':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        #No filter option
        else:
            df = pd.DataFrame.from_records(db.read({}))
        
        #remove last column for processing
        df.drop(columns=['_id'],inplace=True)
        
        return df.to_dict('records')

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#This is the graphing callback and update function
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])


def update_graph(allData):
    
    #Data formatting for scatter plot
    dff = pd.DataFrame.from_dict(allData)
    
    #Scatter plot input parameters
    fig = px.scatter(data_frame=dff, x=dff['breed'], y=dff['outcome_type'])
    
    return fig
       
    
#Map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, derived_virtual_selected_rows):

    dff = df if viewData is None else pd.DataFrame(viewData)
    filtered_animal = None

    #Check if no selection
    if not derived_virtual_selected_rows:
        filtered_animal = dff.iloc[0]
        
    #Use selection display for option
    else:
        filtered_animal = dff.iloc[derived_virtual_selected_rows[0]]

    #setting values based on the .csv input
    latitude = filtered_animal[13]
    longitude = filtered_animal[14]
    animal_breed = filtered_animal[4]
    animal_name = filtered_animal[9]

    return [
        dl.Map(center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            dl.Marker(position=[latitude, longitude], children=[
                
                dl.Tooltip(animal_breed),
                
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)
                ])
            ])
        ])
    ]




app.run_server(debug=True)

Dash app running on http://127.0.0.1:19309/
